## Searching for bouts for a day of ephys recording
- mircophone chan and sync data and bouts are extracted in sglx_preprocess-s_b1253_21-multisess-pouli-ephys

In [1]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2022-10-23 16:21:48,505 root         INFO     Running on pakhi.ucsd.edu


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

### Get the file locations for a session (day) of recordings

In [28]:
reload(et)
sess_par = {'bird': 's_b1555_22',
           'sess': '2022-10-11',
           'sort': 0}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='sglx')

raw_folder = exp_struct['folders']['sglx']

derived_folder = exp_struct['folders']['derived']

bouts_folder = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx')

sess_bouts_file = os.path.join(bouts_folder, 'bout_sess_auto.pickle')
sess_bouts_curated_file = os.path.join(bouts_folder, 'bout_curated.pickle')

#os.makedirs(bouts_folder, exist_ok=True)

In [29]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1555_22',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-10-11',
 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1555_22/2022-10-11/sglx',
 'kwik': '/scratch/earneodo/s_b1555_22/sglx/kwik/2022-10-11',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1555_22/2022-10-11/sglx',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/sglx',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1555_22/sglx/msort/2022-10-11',
 'ksort': '/scratch/earneodo/s_b1555_22/sglx/ksort/2022-10-11/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/sglx/'}

### load concatenated the files of the session

In [30]:
hparams, bout_pd = sb.load_bouts(sess_par['bird'], sess_par['sess'], '', 
                                 derived_folder = 'bouts_sglx', bout_file_key='bout_auto_file')

In [31]:
### list all the source files
sess_wav_files = np.unique(bout_pd['file'].values)
sess_wav_files

array(['/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/sglx/1817_g0/wav_mic.wav'],
      dtype=object)

####  drop the stim recordings, if any
you want to drop the sessions where the stimulus was playing, to avoid counting the stimuli as produced song, since they were recorded in the microphone.

TODO: mask the epochs where stimulus was playing, or tell apart what was stim and what whas coming from the bird.


In [7]:
sess_wav_files[[0, 2, 3]]

array(['/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-12/sglx/0804_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-12/sglx/1525_stim_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-12/sglx/1802_g0/wav_mic.wav'],
      dtype=object)

In [33]:
#stim_epochs = sess_wav_files[[1, 2, 3]]
stim_epochs = []

logger.info('leaving out detected bouts from files {}'.format(list(stim_epochs)))
bout_pd.drop(bout_pd[bout_pd['file'].isin(stim_epochs)].index, inplace=True)


2022-10-23 23:05:33,073 root         INFO     leaving out detected bouts from files []


In [34]:
np.unique(bout_pd['file'].values)

array(['/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/sglx/1817_g0/wav_mic.wav'],
      dtype=object)

In [35]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        138 non-null    int64  
 1   end_ms          138 non-null    int64  
 2   start_sample    138 non-null    int64  
 3   end_sample      138 non-null    int64  
 4   p_step          138 non-null    object 
 5   rms_p           138 non-null    float64
 6   peak_p          138 non-null    float64
 7   bout_check      138 non-null    bool   
 8   file            138 non-null    object 
 9   len_ms          138 non-null    int64  
 10  syl_in          138 non-null    object 
 11  n_syl           138 non-null    int64  
 12  peaks_p         138 non-null    object 
 13  n_peaks         138 non-null    int64  
 14  l_p_ratio       138 non-null    float64
 15  waveform        138 non-null    object 
 16  confusing       138 non-null    bool   
 17  valid_waveform  138 non-null    boo

In [36]:
bout_pd['len_ms']

0       9065
1       3105
2       3200
3       4235
4      14450
       ...  
133     7110
134    18630
135     4590
136     6395
137     5220
Name: len_ms, Length: 138, dtype: int64

In [37]:
np.unique(bout_pd['start_ms']).size

138

## inspect the bouts and curate them

#### visualize one bout

In [38]:
bout_pd.iloc[0]

start_ms                                                     353735
end_ms                                                       362800
start_sample                                               14149400
end_sample                                                 14512000
p_step            [22.70130651950269, 7.358665756051881, 12.0199...
rms_p                                                      4.585068
peak_p                                                   188.695549
bout_check                                                    False
file              /mnt/sphere/speech_bci/derived_data/s_b1555_22...
len_ms                                                         9065
syl_in            [[353735, 353790], [353840, 354040], [354765, ...
n_syl                                                             8
peaks_p           [6, 22, 32, 44, 51, 58, 67, 89, 96, 158, 173, ...
n_peaks                                                          51
l_p_ratio                                       

In [39]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [40]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bout_pd.iloc[0])

    'data': [{'type': 'scatter',
              'uid': 'c8995dfc-6637-4af7-97f9-6…

In [41]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,spectrogram
0,353735,362800,14149400,14512000,"[22.70130651950269, 7.358665756051881, 12.0199...",4.585068,188.695549,False,/mnt/sphere/speech_bci/derived_data/s_b1555_22...,9065,"[[353735, 353790], [353840, 354040], [354765, ...",8,"[6, 22, 32, 44, 51, 58, 67, 89, 96, 158, 173, ...",51,177.745098,"[20, 0, 7, -12, -6, 1, -7, -7, -10, -1, -9, -8...",True,True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,402420,405525,16096800,16221000,"[1.2484208081872763, 3.0529479712645053, 2.528...",4.585068,146.315760,False,/mnt/sphere/speech_bci/derived_data/s_b1555_22...,3105,"[[402420, 402450], [404260, 404325], [405305, ...",4,"[3, 83, 306, 361, 370, 381, 412, 427, 461, 553...",16,194.062500,"[4, 11, 15, 19, 15, 17, 4, -4, 3, 14, 7, 4, 0,...",True,True,True,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


In [42]:
## sort them by duration, which is going to make the curation a bit easier
bout_pd.sort_values('len_ms', ascending=False, inplace=True)
bout_pd.reset_index(drop=True, inplace=True)

In [43]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [44]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_call = None
        self.is_confusing = None
                
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        self.audio = None
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        # is this the best place for this?
        self.bouts_pd['is_call'] = False
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_call = widgets.Checkbox(description='calls')
        self.is_confusing = widgets.Checkbox(description='confusing')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Song", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Noise", button_style='danger',
                                                 icon='wrong')
        self.buttons['Call'] = widgets.Button(description="Calls")
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        
        top_box = widgets.HBox([self.buttons['Prev'], self.buttons['Next']])
        bottom_box = widgets.HBox([self.buttons['Uncheck'], self.buttons['Check'], self.buttons['Call']])
        button_box = widgets.VBox([top_box, bottom_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1, step=1, 
                                        description="Bout candidate index")
        
        
        #self.audio = Audio(data=np.zeros(100), rate=self.s_f)
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing, self.is_call]),
                                  widgets.VBox([self.m_pick])]
                                  )
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_call.observe(self.call_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            #print('prev clicked')
            curr_bout.value -= 1
        
        elif button.description == 'Song':
            #print('check clicked')
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            self.bouts_pd.loc[self.bout_id, 'is_call'] = False
            curr_bout.value += 1
        
        elif button.description == 'Noise':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            self.bouts_pd.loc[self.bout_id, 'is_call'] = False
            curr_bout.value += 1
        
        elif button.description == 'Calls':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            self.bouts_pd.loc[self.bout_id, 'is_call'] = True
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
        #self.bouts_pd.loc[self.bout_id, 'is_call'] = False # if it is 
    
    def call_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'is_call'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_call.value = bool(self.bout_series['is_call'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
                
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
        #self.audio = Audio(data=self.x.flatten(), rate=self.s_f)
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bout_pd)

### save it

In [24]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'sample_rate': 40000,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': <function ceciestunepipe.util.sound.boutsearch.read_wav_chan(wav_path: str, chan_id: int = 0, return_int16=True) -> tuple>,
 'file_order_fun': <function ceciestunepipe.util.sound.boutsearch.sess_file_id(f_path)>,
 'min_segment': 30,
 'min_silence': 3000,
 'min_bout': 3000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 180000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle'}

In [45]:
##save to the curated file path
viz_bout.bouts_pd.to_pickle(sess_bouts_curated_file)
sb.save_auto_bouts(viz_bout.bouts_pd, sess_par, hparams, software='sglx', 
                   bout_file_key='bout_curated_file')

2022-10-23 23:10:38,471 ceciestunepipe.pipeline.searchbout INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/bouts_sglx/bout_checked.pickle
2022-10-23 23:10:40,615 ceciestunepipe.pipeline.searchbout INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/bouts_sglx/bout_search_params.pickle
2022-10-23 23:10:40,616 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/bouts_sglx/bout_search_params.pickle


### summary of where the good bouts are along the session

In [46]:
def give_summary(bpd: pd.DataFrame):
    all_files = np.unique(bpd['file'])
    summary_dict = {f: np.where((bpd['bout_check']==True) & (bpd['file']==f))[0].size for f in all_files}
    return summary_dict

sum_dict = give_summary(viz_bout.bouts_pd)
sum_dict

{'/mnt/sphere/speech_bci/derived_data/s_b1555_22/2022-10-11/sglx/1817_g0/wav_mic.wav': 17}

In [47]:
def sess_bout_summary(bout_pd: pd.DataFrame, ax_dict: dict=None, bouts_type='curated', min_len_ms=10000) -> pd.DataFrame:
    ## make and plot a summary of the bird's bout.
    # get lengths of the bouts
    # get estimate timestamps of bouts
    # plot histogram length of bouts
    # histogram of time of bouts
    bout_sel = (bout_pd['valid']==True) & (bout_pd['len_ms'] > min_len_ms);
    if bouts_type=='curated':
        bout_sel = bout_sel & (bout_pd['bout_check']==True) & (bout_pd['confusing']==False);
    
    # len/time? (when do they sing the longest?)
    logger.info('Number of bouts longer than {} secs: {}'.format(min_len_ms//1000, bout_pd.loc[bout_sel].index.size))
    logger.info('Length of all bouts (minutes): {}'.format(bout_pd.loc[bout_sel, 'len_ms'].values.sum()/60000))
    
    if ax_dict is None:
        bout_pd.loc[bout_sel].hist(column='len_ms')
    return bout_pd

bpd = sess_bout_summary(bout_pd)

2022-10-23 23:10:40,649 root         INFO     Number of bouts longer than 10 secs: 5
2022-10-23 23:10:40,650 root         INFO     Length of all bouts (minutes): 1.9691666666666667
